In [2]:
import numpy as np   
import pandas as pd
import os

import seaborn as sns
import matplotlib.pyplot as plt   
import matplotlib.style
plt.style.use('classic')

import warnings
warnings.filterwarnings("ignore")

In [3]:
#!pip install -q xgboost==0.4a30

In [4]:
import xgboost as xgb

In [5]:
import os
#os.getcwd()
#%cd /content/drive/My\ Drive/
df=pd.read_csv("cleandata.csv")
# Pls use the clean/processed dataset given in the solution ( after all process steps ;just before train/test split) 
#used in the case study for ML- week2

/content/drive/My Drive


In [6]:
df.shape
df.columns
df.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
df.shape
df

,age,cigsPerDay,tot_cholesterol,Diastolic_BP,BMI,heartRate,glucose,Heart-Att,Gender_Male,education_2.0,education_3.0,education_4.0,currentSmoker_1.0,BPMeds_1.0,prevalentStroke_1.0,prevalentHyp_1.0,diabetes_1.0
0,0.184211,0.00,0.325893,0.289530,0.571862,0.575000,0.471154,0,1,0,0,1,0,0,0,0,0
1,0.368421,0.00,0.571429,0.477564,0.660931,0.825000,0.451923,0,0,1,0,0,0,0,0,0,0
2,0.421053,0.40,0.549107,0.460470,0.489372,0.491667,0.336538,0,1,0,0,0,1,0,0,0,0
3,0.763158,0.60,0.459821,0.716880,0.653340,0.325000,0.971154,1,0,0,1,0,1,0,0,1,0
4,0.368421,0.46,0.727679,0.528846,0.376012,0.658333,0.625000,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0.421053,0.40,0.562500,0.323718,0.320344,0.641667,0.644231,0,0,1,0,0,1,0,0,0,0
4236,0.315789,0.30,0.392857,0.580128,0.176619,0.675000,0.490385,0,0,0,0,0,1,0,0,0,0
4237,0.526316,0.00,0.656250,0.511752,0.293522,0.575000,1.000000,0,0,1,0,0,0,0,0,0,0
4238,0.210526,0.00,0.281250,0.768162,0.502530,0.358333,0.375000,0,1,0,1,0,0,0,0,1,0


# New Section

In [8]:
# Copy all the predictor variables into X dataframe
X = df.drop('Heart-Att', axis=1)

# Copy target into the y dataframe.  
y = df['Heart-Att']

In [9]:
# Split X and y into training and test set in 75:25 ratio
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25 , random_state=1)

### **Naive bayes with original sample**

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [11]:
NB_model = GaussianNB()
NB_model.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [12]:
## Performance Matrix on train data set
y_train_predict = NB_model.predict(X_train)
model_score = NB_model.score(X_train, y_train)
print(model_score)
print(metrics.confusion_matrix(y_train, y_train_predict))
print(metrics.classification_report(y_train, y_train_predict))

0.8317610062893082
[[2571  127]
 [ 408   74]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.91      2698
           1       0.37      0.15      0.22       482

    accuracy                           0.83      3180
   macro avg       0.62      0.55      0.56      3180
weighted avg       0.79      0.83      0.80      3180



In [13]:
## Performance Matrix on test data set
y_test_predict = NB_model.predict(X_test)
model_score = NB_model.score(X_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_test_predict))
print(metrics.classification_report(y_test, y_test_predict))

0.8311320754716981
[[853  45]
 [134  28]]
              precision    recall  f1-score   support

           0       0.86      0.95      0.91       898
           1       0.38      0.17      0.24       162

    accuracy                           0.83      1060
   macro avg       0.62      0.56      0.57      1060
weighted avg       0.79      0.83      0.80      1060



# **Logistic regression with original sample**

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
logreg= LogisticRegression()
#logreg= LogisticRegression(class_weight='balanced')
logreg.fit(X_train,y_train)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [16]:
predict_lg_train= logreg.predict(X_train)
print(metrics.confusion_matrix(y_train, predict_lg_train))
print(metrics.classification_report(y_train ,predict_lg_train))

[[1764  934]
 [ 151  331]]
              precision    recall  f1-score   support

           0       0.92      0.65      0.76      2698
           1       0.26      0.69      0.38       482

    accuracy                           0.66      3180
   macro avg       0.59      0.67      0.57      3180
weighted avg       0.82      0.66      0.71      3180



In [17]:
predict_lg_test= logreg.predict(X_test)
print(metrics.confusion_matrix(y_test, predict_lg_test))
print(metrics.classification_report(y_test ,predict_lg_test))

[[597 301]
 [ 51 111]]
              precision    recall  f1-score   support

           0       0.92      0.66      0.77       898
           1       0.27      0.69      0.39       162

    accuracy                           0.67      1060
   macro avg       0.60      0.67      0.58      1060
weighted avg       0.82      0.67      0.71      1060



In [18]:
from sklearn.model_selection import cross_val_score
scores_log = cross_val_score(logreg, X_train, y_train, cv=10).mean()
scores_log

0.6540880503144655

# XGB with orinal sample

In [31]:
y_train.value_counts()
2698/482


5.597510373443983

### **XGB with original sample**

In [32]:

# fit model no training data
xg = xgb.XGBClassifier(learning_rate=0.01,n_estimators=100,max_depth=4, gamma=3, scale_pos_weight=5.59, subsample=.8, colsample_bytree=.7, min_child_weight=10)
xg.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=3,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=10, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=5.59, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [44]:
y_train_predict1 = xg.predict(X_train)
model_score1 = xg.score(X_train, y_train)
print(model_score1)
print(metrics.confusion_matrix(y_train, y_train_predict1))
print(metrics.classification_report(y_train, y_train_predict1))
1## Performance Matrix on test data set
y_test_predict2 = xg.predict(X_test)
model_score2 = xg.score(X_test, y_test)
print(model_score2)
print(metrics.confusion_matrix(y_test, y_test_predict2))
print(metrics.classification_report(y_test, y_test_predict2))

0.7015723270440252
[[1878  820]
 [ 129  353]]
              precision    recall  f1-score   support

           0       0.94      0.70      0.80      2698
           1       0.30      0.73      0.43       482

    accuracy                           0.70      3180
   macro avg       0.62      0.71      0.61      3180
weighted avg       0.84      0.70      0.74      3180

0.6764150943396227
[[609 289]
 [ 54 108]]
              precision    recall  f1-score   support

           0       0.92      0.68      0.78       898
           1       0.27      0.67      0.39       162

    accuracy                           0.68      1060
   macro avg       0.60      0.67      0.58      1060
weighted avg       0.82      0.68      0.72      1060

